In [1]:
import pandas as pd
df = pd.read_excel(r"C:\D\1. Analytics (Data - related)\Business analytics -growth\Design Analysis - Snitch - Business Analyst Growth.xlsx", sheet_name="Data req")
print(df.head())


   INDEX_ SKU_GROUP              CATEGORY OCCASSION_NEW  \
0       1      SKU1              T-Shirts   Casual Wear   
1       2      SKU2  Joggers & Trackpants        Others   
2       3      SKU3                 Jeans   Casual Wear   
3       4      SKU4                Shirts      Elevated   
4       5      SKU5                Shirts   Casual Wear   

            PRINT_DESIGN COLLAR_NEW MATERIAL_NEW  SLEEVE_TYPE            FIT  \
0  Textured, Self-Design       Polo       Cotton  Half Sleeve       Slim Fit   
1                    NaN     Others       Others          NaN            NaN   
2                  Plain     Others       Cotton          NaN     Skinny Fit   
3    Printed, Polka Dots     Spread        Rayon  Full Sleeve       Slim Fit   
4                Printed      Cuban        Rayon  Half Sleeve  Oversized Fit   

       COLOR  ... MONTH4 MONTH5  MONTH6  MONTH7  MONTH8  MONTH9  MONTH10  \
0      Black  ...      5     46      75      14       7       6        8   
1     Maroon

In [2]:
# A look of the data                  
features = ["OCCASSION_NEW", "PRINT_DESIGN", "COLLAR_NEW", "MATERIAL_NEW",
            "SLEEVE_TYPE", "FIT", "COLOR", "CLOSURE", "STYLE"]

# missing values as 'Unknown'
df[features] = df[features].fillna("Unknown")
categories = df["CATEGORY"].dropna().unique()

for category_name in categories:
    print(f"\n{'='*50}\nCategory: {category_name}\n{'='*50}")
    df_cat = df[df["CATEGORY"] == category_name]
    
    for feature in features:
        print(f"\nTop {feature} values influencing sales for {category_name}")
        top_vals = (
            df_cat.groupby(feature)["TOTALS"]
            .sum()
            .sort_values(ascending=False)
            .head
        )
        print(top_vals)


Category: T-Shirts

Top OCCASSION_NEW values influencing sales for T-Shirts
<bound method NDFrame.head of OCCASSION_NEW
Casual Wear    123829
Others             52
Name: TOTALS, dtype: int64>

Top PRINT_DESIGN values influencing sales for T-Shirts
<bound method NDFrame.head of PRINT_DESIGN
Textured, Self-Design                  43482
Plain                                  30667
Printed, Graphic Print                 15899
Textured                                9895
Self-Design, Textured                   3514
Printed                                 3305
Graphic Print                           3257
Colourblocked                           2802
Printed, Abstract                       2261
Embroidered                             1944
Self-Design, Textured, Plain            1123
Tie & Dye                                885
Stripes                                  877
Solid                                    583
Printed, Polka Dots                      524
Printed, Geometric               

In [3]:
!pip install xlsxwriter



In [4]:
# Features to analyze # For part one of assignment 
features = ["OCCASSION_NEW", "PRINT_DESIGN", "COLLAR_NEW", "MATERIAL_NEW", 
            "SLEEVE_TYPE", "FIT", "COLOR", "CLOSURE", "STYLE"] 
 
# Fill missing values with 'Unknown' 
df[features] = df[features].fillna("Unknown") 
 
# Get all categories 
categories = df["CATEGORY"].dropna().unique() 
 
# Prepare Excel writer to save output 
output_path = r"C:\D\1. Analytics (Data - related)\Business analytics -growth\Output_Analysis.xlsx" 
writer = pd.ExcelWriter(output_path, engine='xlsxwriter') 
 
# Step 1: Feature influence by category (percentage of total sales by feature) 
step1_results = [] 
 
# Step 2: Feature values preference (percentage within feature) 
step2_results = [] 
 
# Step 3: Popular feature combinations (top combos per category) 
step3_results = [] 
 
for category in categories: 
    df_cat = df[df["CATEGORY"] == category] 
    total_sales_cat = df_cat["TOTALS"].sum() 
 
    # --- Step 1: Feature influence --- 
    feature_sales = {} 
    feature_sales_pct = {}
    for feature in features: 
        df_filtered = df_cat[df_cat[feature] != "Unknown"] 
        # Sum sales by each feature's values 
        sales_by_feature_val = df_filtered.groupby(feature)["TOTALS"].sum() 
        feature_total = sales_by_feature_val.sum() 
        # Store absolute sales for this feature
        feature_sales[feature] = feature_total
        # % contribution of this feature's total sales to category sales 
        feature_sales_pct[feature] = feature_total / total_sales_cat * 100 if total_sales_cat > 0 else 0 
 
    step1_results.append(pd.DataFrame({ 
        'Category': category, 
        'Feature': list(feature_sales.keys()), 
        'Total_Sales': list(feature_sales.values()),
        'Sales_Contribution_Percent': list(feature_sales_pct.values()) 
    })) 
 
    # --- Step 2: Feature value preference --- 
    for feature in features: 
        sales_by_feature_val = df_cat.groupby(feature)["TOTALS"].sum() 
        feature_total = sales_by_feature_val.sum() 
        # Sort by sales values descending
        sorted_sales = sales_by_feature_val.sort_values(ascending=False)
        # % sales share by each feature value within this feature 
        feature_val_pct = (sorted_sales / feature_total * 100) if feature_total > 0 else sorted_sales * 0
        df_step2 = pd.DataFrame({ 
            'Category': category, 
            'Feature': feature, 
            'Feature_Value': feature_val_pct.index,
            'Total_Sales': sorted_sales.values,
            'Percent_of_Feature_Sales': feature_val_pct.values 
        }) 
        step2_results.append(df_step2)
 
    # --- Step 3: Popular feature combinations --- 
    # Let's find top 10 combos of 3 features (you can adjust which features to combine) 
    combo_features = ["FIT", "MATERIAL_NEW", "COLOR"]  # Example combo set for insight 
    df_cat = df_cat.copy()  
    df_cat['combo'] = df_cat[combo_features].agg('_'.join, axis=1) 
    combo_sales = df_cat.groupby('combo')["TOTALS"].sum().sort_values(ascending=False) 
    combo_total = combo_sales.sum() 
    top_combos = combo_sales.head(10)
    combo_pct = (top_combos / combo_total * 100) if combo_total > 0 else top_combos * 0
    df_step3 = pd.DataFrame({ 
        'Category': category, 
        'Feature_Combination': combo_pct.index, 
        'Total_Sales': top_combos.values,
        'Percent_of_Combo_Sales': combo_pct.values 
    }) 
    step3_results.append(df_step3) 
 
# Write all results to separate sheets 
pd.concat(step1_results).to_excel(writer, sheet_name='Step1_Feature_Influence', index=False) 
pd.concat(step2_results).to_excel(writer, sheet_name='Step2_Feature_Values', index=False) 
pd.concat(step3_results).to_excel(writer, sheet_name='Step3_Top_Combinations', index=False) 
 
# Add a summary sheet with total category sales
category_totals = df.groupby('CATEGORY')['TOTALS'].sum().reset_index()
category_totals.columns = ['Category', 'Total_Sales']
category_totals.to_excel(writer, sheet_name='Category_Sales_Summary', index=False)

writer.close() 
print(f"Output saved to {output_path}")

Output saved to C:\D\1. Analytics (Data - related)\Business analytics -growth\Output_Analysis.xlsx


In [5]:
#For part 2 , 3 of assignment 
import pandas as pd
import numpy as np

# Load the data
file_path = r"C:\D\1. Analytics (Data - related)\Business analytics -growth\Design Analysis - Snitch - Business Analyst Growth.xlsx"
df = pd.read_excel(file_path, sheet_name="Data req")

# Define the feature columns and month columns
feature_columns = ["OCCASSION_NEW", "PRINT_DESIGN", "COLLAR_NEW", "MATERIAL_NEW",
                  "SLEEVE_TYPE", "FIT", "COLOR", "CLOSURE", "STYLE"]
month_columns = [f'MONTH{i}' for i in range(1, 13)]

# Fill missing values with more meaningful labels
df[feature_columns] = df[feature_columns].fillna("Unknown")

# Get unique categories
categories = df["CATEGORY"].dropna().unique()

# Create a list to store all results
all_results = []

# Process each category and feature
print("Generating monthly sales table by category and feature...")
for category in categories:
    # Filter data for this category
    df_cat = df[df["CATEGORY"] == category]
    
    # Process each feature
    for feature in feature_columns:
        # Get unique feature values for this category
        feature_values = df_cat[feature].unique()
        
        # Skip if all are "Unknown"
        if len(feature_values) == 1 and feature_values[0] == "Unknown":
            continue
        
        # Process each feature value
        for feature_value in feature_values:
            # Skip if it's "Unknown"
            if feature_value == "Unknown":
                continue
            
            # Filter data for this feature value
            df_feature = df_cat[df_cat[feature] == feature_value]
            
            # Calculate monthly sales for this category/feature/value combination
            monthly_sales = {}
            for i, month in enumerate(month_columns, 1):
                monthly_sales[f"Month {i}"] = df_feature[month].sum()
            
            # Create a row for this combination
            result_row = {
                "Category": category,
                "Feature": feature,
                "Feature Value": feature_value
            }
            
            # Add monthly sales data
            result_row.update(monthly_sales)
            
            # Add to results
            all_results.append(result_row)

# Create DataFrame from results
if all_results:
    results_df = pd.DataFrame(all_results)
    
    # Create Excel writer for output
    output_path = r"C:\D\1. Analytics (Data - related)\Business analytics -growth\Monthly_Sales_By_Feature.xlsx"
    
    # Save to Excel
    try:
        with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
            results_df.to_excel(writer, sheet_name='Monthly_Sales', index=False)
            
            # Get the xlsxwriter workbook and worksheet objects
            workbook = writer.book
            worksheet = writer.sheets['Monthly_Sales']
            
            # Create a format for the header row
            header_format = workbook.add_format({
                'bold': True,
                'text_wrap': True,
                'valign': 'top',
                'fg_color': '#D7E4BC',
                'border': 1
            })
            
            # Write the column headers with the defined format
            for col_num, value in enumerate(results_df.columns.values):
                worksheet.write(0, col_num, value, header_format)
            
            # Set column widths
            worksheet.set_column('A:A', 20)  # Category
            worksheet.set_column('B:B', 15)  # Feature
            worksheet.set_column('C:C', 25)  # Feature Value
            worksheet.set_column('D:O', 10)  # Months
        
        print(f"Table generated successfully! Saved to: {output_path}")
    except Exception as e:
        print(f"Error saving to Excel: {e}")
        
        # Fallback: save without formatting
        results_df.to_excel(output_path, index=False)
        print(f"Table saved with basic formatting to: {output_path}")
else:
    print("No data found for the specified criteria.")

Generating monthly sales table by category and feature...
Table generated successfully! Saved to: C:\D\1. Analytics (Data - related)\Business analytics -growth\Monthly_Sales_By_Feature.xlsx


In [6]:
import pandas as pd
import numpy as np

# Load the data
file_path = r"C:\D\1. Analytics (Data - related)\Business analytics -growth\Design Analysis - Snitch - Business Analyst Growth.xlsx"
df = pd.read_excel(file_path, sheet_name="Data req")

# Define the feature columns and month columns
feature_columns = ["OCCASSION_NEW", "PRINT_DESIGN", "COLLAR_NEW", "MATERIAL_NEW",
                  "SLEEVE_TYPE", "FIT", "COLOR", "CLOSURE", "STYLE"]
month_columns = [f'MONTH{i}' for i in range(1, 13)]

# Fill missing values with more meaningful labels
df[feature_columns] = df[feature_columns].fillna("Unknown")

# Get unique categories
categories = df["CATEGORY"].dropna().unique()

# Create a list to store all results
all_results = []

# Process each category and feature
print("Generating monthly sales table by category and feature...")
for category in categories:
    # Filter data for this category
    df_cat = df[df["CATEGORY"] == category]
    
    # Process each feature
    for feature in feature_columns:
        # Get unique feature values for this category
        feature_values = df_cat[feature].unique()
        
        # Skip if all are "Unknown"
        if len(feature_values) == 1 and feature_values[0] == "Unknown":
            continue
        
        # Process each feature value
        for feature_value in feature_values:
            # Skip if it's "Unknown"
            if feature_value == "Unknown":
                continue
            
            # Filter data for this feature value
            df_feature = df_cat[df_cat[feature] == feature_value]
            
            # Calculate monthly sales for this category/feature/value combination
            monthly_sales = {}
            for i, month in enumerate(month_columns, 1):
                monthly_sales[f"Month {i}"] = df_feature[month].sum()
            
            # Create a row for this combination
            result_row = {
                "Category": category,
                "Feature": feature,
                "Feature Value": feature_value
            }
            
            # Add monthly sales data
            result_row.update(monthly_sales)
            
            # Add to results
            all_results.append(result_row)

# Create DataFrame from results
if all_results:
    results_df = pd.DataFrame(all_results)
    
    # Create Excel writer for output
    output_path = r"C:\D\1. Analytics (Data - related)\Business analytics -growth\Monthly_Sales_By_Feature.xlsx"
    
    # Save to Excel
    try:
        with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
            results_df.to_excel(writer, sheet_name='Monthly_Sales', index=False)
            
            # Get the xlsxwriter workbook and worksheet objects
            workbook = writer.book
            worksheet = writer.sheets['Monthly_Sales']
            
            # Create a format for the header row
            header_format = workbook.add_format({
                'bold': True,
                'text_wrap': True,
                'valign': 'top',
                'fg_color': '#D7E4BC',
                'border': 1
            })
            
            # Write the column headers with the defined format
            for col_num, value in enumerate(results_df.columns.values):
                worksheet.write(0, col_num, value, header_format)
            
            # Set column widths
            worksheet.set_column('A:A', 20)  # Category
            worksheet.set_column('B:B', 15)  # Feature
            worksheet.set_column('C:C', 25)  # Feature Value
            worksheet.set_column('D:O', 10)  # Months
        
        print(f"Table generated successfully! Saved to: {output_path}")
    except Exception as e:
        print(f"Error saving to Excel: {e}")
        
        # Fallback: save without formatting
        results_df.to_excel(output_path, index=False)
        print(f"Table saved with basic formatting to: {output_path}")
else:
    print("No data found for the specified criteria.")

Generating monthly sales table by category and feature...
Table generated successfully! Saved to: C:\D\1. Analytics (Data - related)\Business analytics -growth\Monthly_Sales_By_Feature.xlsx


In [9]:
import pandas as pd
import numpy as np

# Load the data
file_path = r"C:\D\1. Analytics (Data - related)\Business analytics -growth\Design Analysis - Snitch - Business Analyst Growth.xlsx"
df = pd.read_excel(file_path, sheet_name="Data req")

# Features to analyze 
features = ["OCCASSION_NEW", "PRINT_DESIGN", "COLLAR_NEW", "MATERIAL_NEW", 
            "SLEEVE_TYPE", "FIT", "COLOR", "CLOSURE", "STYLE"] 

# Fill missing values with 'Unknown' 
df[features] = df[features].fillna("Unknown") 

# Get all categories 
categories = df["CATEGORY"].dropna().unique() 

# Prepare Excel writer to save output 
output_path = r"C:\D\1. Analytics (Data - related)\Business analytics -growth\Output_Analysis.xlsx" 
writer = pd.ExcelWriter(output_path, engine='xlsxwriter') 

# Step 1: Feature influence by category (percentage of total sales by feature) 
step1_results = [] 

# Step 2: Feature values preference (percentage within feature) 
step2_results = [] 

# Step 3: Popular feature combinations (top combos per category) 
step3_results = [] 

# Step 4: Feature composition recommendations
step4_results = []

for category in categories: 
    df_cat = df[df["CATEGORY"] == category] 
    total_sales_cat = df_cat["TOTALS"].sum() 

    # --- Step 1: Feature influence --- 
    feature_sales = {} 
    feature_sales_pct = {}
    for feature in features: 
        df_filtered = df_cat[df_cat[feature] != "Unknown"] 
        # Sum sales by each feature's values 
        sales_by_feature_val = df_filtered.groupby(feature)["TOTALS"].sum() 
        feature_total = sales_by_feature_val.sum() 
        # Store absolute sales for this feature
        feature_sales[feature] = feature_total
        # % contribution of this feature's total sales to category sales 
        feature_sales_pct[feature] = feature_total / total_sales_cat * 100 if total_sales_cat > 0 else 0 
 
    step1_results.append(pd.DataFrame({ 
        'Category': category, 
        'Feature': list(feature_sales.keys()), 
        'Total_Sales': list(feature_sales.values()),
        'Sales_Contribution_Percent': list(feature_sales_pct.values()) 
    })) 
 
    # --- Step 2: Feature value preference --- 
    for feature in features: 
        sales_by_feature_val = df_cat.groupby(feature)["TOTALS"].sum() 
        feature_total = sales_by_feature_val.sum() 
        # Sort by sales values descending
        sorted_sales = sales_by_feature_val.sort_values(ascending=False)
        # % sales share by each feature value within this feature 
        feature_val_pct = (sorted_sales / feature_total * 100) if feature_total > 0 else sorted_sales * 0
        df_step2 = pd.DataFrame({ 
            'Category': category, 
            'Feature': feature, 
            'Feature_Value': feature_val_pct.index,
            'Total_Sales': sorted_sales.values,
            'Percent_of_Feature_Sales': feature_val_pct.values 
        }) 
        step2_results.append(df_step2)
 
    # --- Step 3: Popular feature combinations --- 
    # Let's find top 10 combos of 3 features (you can adjust which features to combine) 
    combo_features = ["FIT", "MATERIAL_NEW", "COLOR"]  # Example combo set for insight 
    df_cat = df_cat.copy()  
    df_cat['combo'] = df_cat[combo_features].agg('_'.join, axis=1) 
    combo_sales = df_cat.groupby('combo')["TOTALS"].sum().sort_values(ascending=False) 
    combo_total = combo_sales.sum() 
    top_combos = combo_sales.head(10)
    combo_pct = (top_combos / combo_total * 100) if combo_total > 0 else top_combos * 0
    df_step3 = pd.DataFrame({ 
        'Category': category, 
        'Feature_Combination': combo_pct.index, 
        'Total_Sales': top_combos.values,
        'Percent_of_Combo_Sales': combo_pct.values 
    }) 
    step3_results.append(df_step3)
    
    # --- Step 4: Feature composition recommendations ---
    # Get top features for this category (features with highest sales contribution)
    if len(step1_results) > 0:
        category_features = step1_results[-1]
        # Sort by sales contribution and get top features (e.g., top 3)
        important_features = category_features.sort_values('Sales_Contribution_Percent', ascending=False)['Feature'].head(3).tolist()
        
        for feature in important_features:
            # Get composition recommendations based on actual sales distribution
            sales_by_feature_val = df_cat.groupby(feature)["TOTALS"].sum()
            feature_total = sales_by_feature_val.sum()
            
            if feature_total > 0:
                feature_val_pct = (sales_by_feature_val / feature_total * 100).sort_values(ascending=False)
                
                # Create recommendation DataFrame
                df_step4 = pd.DataFrame({
                    'Category': category,
                    'Feature': feature,
                    'Feature_Value': feature_val_pct.index,
                    'Recommended_Composition_Percent': feature_val_pct.values.round(2)
                })
                
                step4_results.append(df_step4)
    
    # Additional: Print top feature values influencing sales for each category
    print(f"\n{'='*50}\nCategory: {category}\n{'='*50}")
    
    for feature in features:
        print(f"\nTop {feature} values influencing sales for {category}")
        top_vals = (
            df_cat.groupby(feature)["TOTALS"]
            .sum()
            .sort_values(ascending=False)
            .head(5)  # Show top 5
        )
        print(top_vals)

# Write all results to separate sheets 
pd.concat(step1_results).to_excel(writer, sheet_name='Step1_Feature_Influence', index=False) 
pd.concat(step2_results).to_excel(writer, sheet_name='Step2_Feature_Values', index=False) 
pd.concat(step3_results).to_excel(writer, sheet_name='Step3_Top_Combinations', index=False)

# Write composition recommendations to a new sheet
if step4_results:
    pd.concat(step4_results).to_excel(writer, sheet_name='Step4_CR', index=False)
 
# Add a summary sheet with total category sales
category_totals = df.groupby('CATEGORY')['TOTALS'].sum().reset_index()
category_totals.columns = ['Category', 'Total_Sales']
category_totals.to_excel(writer, sheet_name='Category_Sales_Summary', index=False)

writer.close() 
print(f"Output saved to {output_path}")


Category: T-Shirts

Top OCCASSION_NEW values influencing sales for T-Shirts
OCCASSION_NEW
Casual Wear    123829
Others             52
Name: TOTALS, dtype: int64

Top PRINT_DESIGN values influencing sales for T-Shirts
PRINT_DESIGN
Textured, Self-Design     43482
Plain                     30667
Printed, Graphic Print    15899
Textured                   9895
Self-Design, Textured      3514
Name: TOTALS, dtype: int64

Top COLLAR_NEW values influencing sales for T-Shirts
COLLAR_NEW
Polo        75533
Round       44021
Spread       1630
Others       1178
Baseball      600
Name: TOTALS, dtype: int64

Top MATERIAL_NEW values influencing sales for T-Shirts
MATERIAL_NEW
Cotton       90365
Polyester    20419
Nylon         5697
Rayon         4196
Others        2038
Name: TOTALS, dtype: int64

Top SLEEVE_TYPE values influencing sales for T-Shirts
SLEEVE_TYPE
Half Sleeve     107165
Full Sleeve      10906
Elbow Sleeve      5069
Sleeveless         689
Unknown             52
Name: TOTALS, dtype: int64
